#Задание 1. Решите задачу классификации ансамблевыми алгоритмами бустинга, бэгинга. Напишите комментарии к программному коду.  

-- Если сможете для выполнения заданий используйте свои данные для обучения, если не получится используйте учебные

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set_style("whitegrid")
plt.style.use("fivethirtyeight")

In [ ]:
import time # Импорт модуля time для работы с функциями времени

from google.colab import drive # Импорт специального модуля для работы с Google Drive в среде Google Colab

drive.mount('/content/gdrive')



In [ ]:
df = pd.read_csv("/content/gdrive/My Drive/КАФЕДРА/МАШИННОЕ/diabetes.csv")
df.head()

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df.columns

In [ ]:
# How many missing zeros are mising in each feature
feature_columns = [
    'Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness',
    'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age'
]

for column in feature_columns:
    print("============================================")
    print(f"{column} ==> Missing zeros : {len(df.loc[df[column] == 0])}")

In [ ]:
from sklearn.impute import SimpleImputer


fill_values = SimpleImputer(missing_values=0, strategy="mean", copy=False)
df[feature_columns] = fill_values.fit_transform(df[feature_columns])

for column in feature_columns:
    print("============================================")
    print(f"{column} ==> Missing zeros : {len(df.loc[df[column] == 0])}")

In [ ]:
from sklearn.model_selection import train_test_split


X = df[feature_columns]
y = df.Outcome

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report


def evaluate(model, X_train, X_test, y_train, y_test):
    y_test_pred = model.predict(X_test)
    y_train_pred = model.predict(X_train)

    print("TRAINIG RESULTS: \n===============================")
    clf_report = pd.DataFrame(classification_report(y_train, y_train_pred, output_dict=True))
    print(f"CONFUSION MATRIX:\n{confusion_matrix(y_train, y_train_pred)}")
    print(f"ACCURACY SCORE:\n{accuracy_score(y_train, y_train_pred):.4f}")
    print(f"CLASSIFICATION REPORT:\n{clf_report}")

    print("TESTING RESULTS: \n===============================")
    clf_report = pd.DataFrame(classification_report(y_test, y_test_pred, output_dict=True))
    print(f"CONFUSION MATRIX:\n{confusion_matrix(y_test, y_test_pred)}")
    print(f"ACCURACY SCORE:\n{accuracy_score(y_test, y_test_pred):.4f}")
    print(f"CLASSIFICATION REPORT:\n{clf_report}")

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

base_clf = DecisionTreeClassifier()
model = BaggingClassifier(estimator=base_clf, n_estimators=10)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html

sklearn.ensemble.BaggingClassifier ( estimator = None , n_estimators = 10 , * , max_samples = 1.0 , max_features = 1.0 , bootstrap = True , bootstrap_features = False , oob_score = False , warm_start = False , n_jobs = None , random_state = None , verbose = 0 )



n_estimators int, по умолчанию=10 Число базовых оценок в ансамбле.
max_samples int или float, по умолчанию=1.0
Количество выборок, которые необходимо извлечь из X для обучения каждой базовой оценки
max_features int или float, по умолчанию=1.0
Количество признаков, которые необходимо извлечь из X для обучения каждого базового оценщика
bootstrap bool, по умолчанию = True
Отбор проб с заменой. Если «Ложь», отбор проб производится без замены.
bootstrap_features bool, по умолчанию = False
random_state int, экземпляр RandomState или None, по умолчанию = None
Управляет случайной подвыборкой исходного набора данных


In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier()
bagging_clf = BaggingClassifier(estimator=tree, n_estimators=1500, random_state=42)
bagging_clf.fit(X_train, y_train)



In [ ]:
evaluate(bagging_clf, X_train, X_test, y_train, y_test)

In [ ]:
scores = {
    'Bagging Classifier': {
        'Train': accuracy_score(y_train, bagging_clf.predict(X_train)),
        'Test': accuracy_score(y_test, bagging_clf.predict(X_test)),
    },
}

In [ ]:
scores

**Extra **

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

ex_tree_clf = ExtraTreesClassifier(n_estimators=1000, max_features=7, random_state=42)
ex_tree_clf.fit(X_train, y_train)
evaluate(ex_tree_clf, X_train, X_test, y_train, y_test)

In [ ]:
scores['Extra Tree'] = {
        'Train': accuracy_score(y_train, ex_tree_clf.predict(X_train)),
        'Test': accuracy_score(y_test, ex_tree_clf.predict(X_test)),
    }
scores

**Boosting Algorithms**

In [ ]:
AdaBoost

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html

sklearn.ensemble.AdaBoostClassifier(estimator=None, *, n_estimators=50, learning_rate=1.0, algorithm='deprecated', random_state=None)[source]


In [ ]:
from sklearn.ensemble import AdaBoostClassifier

ada_boost_clf = AdaBoostClassifier(n_estimators=30)
ada_boost_clf.fit(X_train, y_train)
evaluate(ada_boost_clf, X_train, X_test, y_train, y_test)

In [ ]:
scores['AdaBoost'] = {
        'Train': accuracy_score(y_train, ada_boost_clf.predict(X_train)),
        'Test': accuracy_score(y_test, ada_boost_clf.predict(X_test)),
    }
scores

**Stochastic Gradient Boosting**

In [ ]:
sklearn.ensemble.GradientBoostingClassifier(*, loss='log_loss', learning_rate=0.1, n_estimators=100, subsample=1.0, criterion='friedman_mse', min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_depth=3, min_impurity_decrease=0.0, init=None, random_state=None, max_features=None, verbose=0, max_leaf_nodes=None, warm_start=False, validation_fraction=0.1, n_iter_no_change=None, tol=0.0001, ccp_alpha=0.0)

In [ ]:
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

grad_boost_clf = GradientBoostingClassifier(n_estimators=100, random_state=42)
grad_boost_clf.fit(X_train, y_train)
evaluate(grad_boost_clf, X_train, X_test, y_train, y_test)

In [ ]:
scores['Gradient Boosting'] = {
        'Train': accuracy_score(y_train, grad_boost_clf.predict(X_train)),
        'Test': accuracy_score(y_test, grad_boost_clf.predict(X_test)),
    }
scores

**Voting Ensemble**
****
****

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

estimators = []
log_reg = LogisticRegression(solver='liblinear')
estimators.append(('Logistic', log_reg))

tree = DecisionTreeClassifier()
estimators.append(('Tree', tree))

svm_clf = SVC(gamma='scale')
estimators.append(('SVM', svm_clf))

voting = VotingClassifier(estimators=estimators)
voting.fit(X_train, y_train)

evaluate(voting, X_train, X_test, y_train, y_test)

In [ ]:
scores['Voting'] = {
        'Train': accuracy_score(y_train, voting.predict(X_train)),
        'Test': accuracy_score(y_test, voting.predict(X_test)),
    }
scores

**Model Comparison¶**

In [ ]:
scores_df = pd.DataFrame(scores)

scores_df.plot(kind='barh', figsize=(15, 8))

#Задание 2. Выполните классификацию данных методом стекинга. Напишите комментарии к программному коду.


-- Если сможете для выполнения заданий используйте свои данные для обучения, если не получится используйте учебные


In [ ]:
import pandas as pd

# Read the training datasets
X_train_original = pd.read_csv('/content/gdrive/My Drive/КАФЕДРА/МАШИННОЕ/X_train_original.csv')
X_train_original = X_train_original.set_index('Unnamed: 0')

X_train_transformed = pd.read_csv('/content/gdrive/My Drive/КАФЕДРА/МАШИННОЕ/X_train_transformed.csv')
X_train_transformed = X_train_transformed.set_index('Unnamed: 0')

X_train_discretized = pd.read_csv('/content/gdrive/My Drive/КАФЕДРА/МАШИННОЕ/X_train_discretized.csv')
X_train_discretized = X_train_discretized.set_index('Unnamed: 0')

# Read the test datasets
X_test_original = pd.read_csv('/content/gdrive/My Drive/КАФЕДРА/МАШИННОЕ/X_test_original.csv')
X_test_original = X_test_original.set_index('Unnamed: 0')

X_test_transformed = pd.read_csv('/content/gdrive/My Drive/КАФЕДРА/МАШИННОЕ/X_test_transformed.csv')
X_test_transformed = X_test_transformed.set_index('Unnamed: 0')

X_test_discretized = pd.read_csv('/content/gdrive/My Drive/КАФЕДРА/МАШИННОЕ/X_test_discretized.csv')
X_test_discretized = X_test_discretized.set_index('Unnamed: 0')

# Read the target variable for training and testing
y_train = pd.read_csv('/content/gdrive/My Drive/КАФЕДРА/МАШИННОЕ/y_train.csv')
y_train = y_train.set_index('Unnamed: 0').squeeze()

y_test = pd.read_csv('/content/gdrive/My Drive/КАФЕДРА/МАШИННОЕ/y_test.csv')
y_test = y_test.set_index('Unnamed: 0').squeeze()

In [ ]:
X_train_original

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report

logistic = LogisticRegression(penalty=None, C=1.0, fit_intercept=True, class_weight='balanced', l1_ratio=None)
logistic.fit(X_train_transformed, y_train)

y_pred_train = logistic.predict(X_train_transformed)
y_pred_test = logistic.predict(X_test_transformed)
y_prob_test = logistic.predict_proba (X_test_transformed)

cm_train = confusion_matrix(y_train, y_pred_train)
report_train = classification_report(y_train, y_pred_train)

cm_test = confusion_matrix(y_test, y_pred_test)
report_test = classification_report(y_test, y_pred_test)

print("Evaluation the Model on Training Set")
print(f"Confusion Matrix:\n{cm_train}")
print(f"Classification Report:\n{report_train}")
print("-"*80)
print("Evaluation the Model on Testing Set")
print(f"Confusion Matrix:\n{cm_test}")
print(f"Classification Report:\n{report_test}")

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import confusion_matrix, classification_report

bag_clf = BaggingClassifier(estimator=logistic, n_estimators=100, max_samples=1.0, max_features=1.0,
                            bootstrap=True, bootstrap_features=False, n_jobs=-1, random_state=111)

bag_clf.fit(X_train_transformed, y_train)

y_pred_train = bag_clf.predict(X_train_transformed)
y_pred_test = bag_clf.predict(X_test_transformed)
y_prob_test = bag_clf.predict_proba (X_test_transformed)

cm_train = confusion_matrix(y_train, y_pred_train)
report_train = classification_report(y_train, y_pred_train)

cm_test = confusion_matrix(y_test, y_pred_test)
report_test = classification_report(y_test, y_pred_test)

print("Evaluation the Model on Training Set")
print(f"Confusion Matrix:\n{cm_train}")
print(f"Classification Report:\n{report_train}")
print("-"*80)
print("Evaluation the Model on Testing Set")
print(f"Confusion Matrix:\n{cm_test}")
print(f"Classification Report:\n{report_test}")

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report

rf_clf = RandomForestClassifier(n_estimators=100, criterion='gini', max_depth=4, min_samples_split=2,
                                min_samples_leaf=1, max_features='sqrt', bootstrap=True, n_jobs=-1,
                                random_state=111, class_weight='balanced', max_samples=None)

rf_clf.fit(X_train_transformed, y_train)

y_pred_train = rf_clf.predict(X_train_transformed)
y_pred_test = rf_clf.predict(X_test_transformed)
y_prob_test = rf_clf.predict_proba (X_test_transformed)

cm_train = confusion_matrix(y_train, y_pred_train)
report_train = classification_report(y_train, y_pred_train)

cm_test = confusion_matrix(y_test, y_pred_test)
report_test = classification_report(y_test, y_pred_test)

print("Evaluation the Model on Training Set")
print(f"Confusion Matrix:\n{cm_train}")
print(f"Classification Report:\n{report_train}")
print("-"*80)
print("Evaluation the Model on Testing Set")
print(f"Confusion Matrix:\n{cm_test}")
print(f"Classification Report:\n{report_test}")

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix, classification_report

adabo_clf = AdaBoostClassifier(estimator=logistic, n_estimators=10, learning_rate=0.01, algorithm='SAMME', random_state=111)
adabo_clf.fit(X_train_transformed, y_train)

y_pred_train = adabo_clf.predict(X_train_transformed)
y_pred_test = adabo_clf.predict(X_test_transformed)
y_prob_test = adabo_clf.predict_proba (X_test_transformed)

cm_train = confusion_matrix(y_train, y_pred_train)
report_train = classification_report(y_train, y_pred_train)

cm_test = confusion_matrix(y_test, y_pred_test)
report_test = classification_report(y_test, y_pred_test)

print("Evaluation the Model on Training Set")
print(f"Confusion Matrix:\n{cm_train}")
print(f"Classification Report:\n{report_train}")
print("-"*80)
print("Evaluation the Model on Testing Set")
print(f"Confusion Matrix:\n{cm_test}")
print(f"Classification Report:\n{report_test}")

**Stacking**

**Fit the Base Classifiers**

In [ ]:
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(criterion='gini', max_depth=2, min_samples_split=10,
                             min_samples_leaf=5, random_state=111, class_weight="balanced")

In [ ]:
from sklearn.linear_model import LogisticRegression

logistic = LogisticRegression(penalty=None, C=1.0, fit_intercept=True, class_weight='balanced', l1_ratio=None)

In [ ]:
from sklearn.svm import LinearSVC

lsvm = LinearSVC(penalty='l1', loss='squared_hinge', C=0.5, multi_class='ovr',
                 fit_intercept=True, class_weight='balanced', dual=False, random_state=111, max_iter=1000000)

In [ ]:
from sklearn.naive_bayes import GaussianNB

nb = GaussianNB(priors=None)

In [ ]:
from sklearn.neural_network import MLPClassifier

ann = MLPClassifier(hidden_layer_sizes=(20, 10), activation='tanh', solver='adam', alpha=0, batch_size=20,
                learning_rate='constant', learning_rate_init=0.01, max_iter=100000, random_state=111, n_iter_no_change=20)

**Fit the Stacking Classifier**

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import confusion_matrix, classification_report

stack_clf = StackingClassifier([("tree",tree), ("logistic",logistic), ("lsvm",lsvm), ("nb",nb)],
                               final_estimator=nb, cv=5, stack_method='auto', n_jobs=-1, passthrough=True, verbose=0)

# Fit the model
stack_clf.fit(X_train_transformed, y_train)

# Make predictions
y_pred_train = stack_clf.predict(X_train_transformed)
y_pred_test = stack_clf.predict(X_test_transformed)
#y_prob_test = stack_clf.predict_proba(X_test_discretized)

# Evaluate the model on the training set
cm_train = confusion_matrix(y_train, y_pred_train)
report_train = classification_report(y_train, y_pred_train)

# Evaluate the model on the testing set
cm_test = confusion_matrix(y_test, y_pred_test)
report_test = classification_report(y_test, y_pred_test)

# Print the evaluation results
print("Evaluation of the Model on Training Set")
print(f"Confusion Matrix:\n{cm_train}")
print(f"Classification Report:\n{report_train}")
print("-"*80)
print("Evaluation of the Model on Testing Set")
print(f"Confusion Matrix:\n{cm_test}")
print(f"Classification Report:\n{report_test}")

In [ ]:
stack_clf

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt

# Define individual classifiers
tree = DecisionTreeClassifier(criterion='gini', max_depth=2, min_samples_split=10,
                              min_samples_leaf=5, random_state=111, class_weight="balanced")
logistic = LogisticRegression(penalty=None, C=1.0, fit_intercept=True, class_weight='balanced', l1_ratio=None)
lsvm = LinearSVC(penalty='l1', loss='squared_hinge', C=0.5, multi_class='ovr',
                 fit_intercept=True, class_weight='balanced', dual=False, random_state=111, max_iter=1000000)
nb = GaussianNB(priors=None)
ann = MLPClassifier(hidden_layer_sizes=(20, 10), activation='tanh', solver='adam', alpha=0, batch_size=20,
                    learning_rate='constant', learning_rate_init=0.01, max_iter=100000, random_state=111, n_iter_no_change=20)

# Fit each individual model
tree.fit(X_train_transformed, y_train)
logistic.fit(X_train_transformed, y_train)
lsvm.fit(X_train_transformed, y_train)
nb.fit(X_train_transformed, y_train)
ann.fit(X_train_transformed, y_train)

# Create a stacked classifier
stack_clf = StackingClassifier(estimators=[("tree", tree), ("logistic", logistic),
                                           ("lsvm", lsvm), ("nb", nb)],
                               final_estimator=nb, cv=5, stack_method='auto', n_jobs=-1, passthrough=True, verbose=0)

# Fit the stacked classifier
stack_clf.fit(X_train_transformed, y_train)

# Compute ROC curves and ROC area for each classifier
classifiers = {
    "Decision Tree": tree,
    "Logistic Regression": logistic,
    "Linear SVM": lsvm,
    "Gaussian Naive Bayes": nb,
    "MLP Classifier": ann,
    "Stacked Classifier": stack_clf
}

# Initialize lists to store fpr, tpr, and roc_auc for each classifier
fpr = {}
tpr = {}
roc_auc = {}

# Compute ROC curves and ROC area for each classifier
for name, clf in classifiers.items():
    if name == "Linear SVM":
        y_pred_decision = clf.decision_function(X_test_transformed)
        fpr[name], tpr[name], _ = roc_curve(y_test, y_pred_decision)
    elif name == "Stacked Classifier":
        y_pred_proba = stack_clf.predict_proba(X_test_transformed)[:, 1]
        fpr[name], tpr[name], _ = roc_curve(y_test, y_pred_proba)
    else:
        y_pred_proba = clf.predict_proba(X_test_transformed)[:, 1]
        fpr[name], tpr[name], _ = roc_curve(y_test, y_pred_proba)

    roc_auc[name] = roc_auc_score(y_test, y_pred_proba)

# Plot ROC curves
plt.figure(figsize=(6, 4))
for name in classifiers:
    plt.plot(fpr[name], tpr[name], lw=2, label=f'{name} (AUC = {roc_auc[name]:.2f})')

plt.plot([0, 1], [0, 1], color='gray', lw=1, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves on Test Data')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()